<a href="https://colab.research.google.com/github/ickma2311/mycolab/blob/main/cuda/vector_add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cupy-cuda12x


In [ ]:
kernal_code="""
extern "C"
__global__ void vector_add(float *a, float *b, float *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

"""

In [ ]:
import cupy as cp
import numpy as np


In [ ]:
def calculated_expected_resultes(n):
  h_a=np.random.rand(n).astype(np.float32)
  h_b=np.random.rand(n).astype(np.float32)
  h_c=h_a+h_b
  return h_a,h_b,h_c

In [ ]:
n=100000000
h_a,h_b,expected_c=calculated_expected_resultes(n)

In [ ]:
%%timeit
h_a+h_b
# how many time CPU spends

220 ms ± 39.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
d_a=cp.asarray(h_a)
d_b=cp.asarray(h_b)
d_c=cp.zeros_like(d_a)

In [ ]:
cp.cuda.runtime.deviceSynchronize()

In [ ]:
moudle=cp.RawModule(code=kernal_code)
vector_add_kernel=moudle.get_function('vector_add')

In [ ]:
threads_per_block=256
import math
blocks_per_grid=math.ceil(n/threads_per_block)

In [ ]:
%%time
vector_add_kernel(
        (blocks_per_grid,),          # Grid dimensions (tuple)
        (threads_per_block,),       # Block dimensions (tuple)
        (d_a, d_b, d_c, np.int32(n)) # Arguments (tuple)
    )
cp.cuda.runtime.deviceSynchronize()

CPU times: user 4.16 ms, sys: 347 µs, total: 4.51 ms
Wall time: 4.7 ms


In [ ]:
h_c=d_c.get()

In [ ]:
np.testing.assert_allclose(h_c, expected_c, atol=1e-5, rtol=0)
